## L'extraction des données du site web

### Creation de ma base de données pour son stockage



In [1]:
# import sqlite3

# # --- Connexion à la base de données ---
# conn = sqlite3.connect('books.db')
# cursor = conn.cursor()

# # --- Création de la table (si elle n'existe pas) ---
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS books (
#         title TEXT,
#         price TEXT,
#         Description TEXT
#     )''')
# conn.commit()


In [16]:
!pip install "pinecone[grpc]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.1 MB/s eta 0:00:00


In [37]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import numpy as np
import pandas as pd
import uuid
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns


In [32]:


pc = Pinecone(api_key="pcsk_5d1o81_B9WBR6tFQrcurwSSb9mjmha5qSd4U6LBffk8Ew2pFcYB9tjXi9TTxtdTxVAoN8r")

index_name = "eeia-1"
pc.delete_index(index_name)
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        vector_type="dense",
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ),
        deletion_protection="disabled",
        tags={
            "environment": "development"
        }
    )

In [33]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(type(embeddings))
print(embeddings.size)

<class 'numpy.ndarray'>
768


In [34]:
texts = [
    { "_id": "rec1", "chunk_text": "The Eiffel Tower was completed in 1889 and stands in Paris, France.", "category": "history" },
    { "_id": "rec2", "chunk_text": "Photosynthesis allows plants to convert sunlight into energy.", "category": "science" },
    { "_id": "rec3", "chunk_text": "Albert Einstein developed the theory of relativity.", "category": "science" },
    { "_id": "rec4", "chunk_text": "The mitochondrion is often called the powerhouse of the cell.", "category": "biology" },
    { "_id": "rec5", "chunk_text": "Shakespeare wrote many famous plays, including Hamlet and Macbeth.", "category": "literature" },
    { "_id": "rec6", "chunk_text": "Water boils at 100°C under standard atmospheric pressure.", "category": "physics" },
    { "_id": "rec7", "chunk_text": "The Great Wall of China was built to protect against invasions.", "category": "history" },
    { "_id": "rec8", "chunk_text": "Honey never spoils due to its low moisture content and acidity.", "category": "food science" },
    { "_id": "rec9", "chunk_text": "The speed of light in a vacuum is approximately 299,792 km/s.", "category": "physics" },
    { "_id": "rec10", "chunk_text": "Newton's laws describe the motion of objects.", "category": "physics" },
    { "_id": "rec11", "chunk_text": "The human brain has approximately 86 billion neurons.", "category": "biology" },
    { "_id": "rec12", "chunk_text": "The Amazon Rainforest is one of the most biodiverse places on Earth.", "category": "geography" },
    { "_id": "rec13", "chunk_text": "Black holes have gravitational fields so strong that not even light can escape.", "category": "astronomy" },
]

dense_index = pc.Index(index_name)
for text in texts:
    print(text)
    # Target the index

    # Upsert the records into a namespace
    dense_index.upsert(vectors=[
        {
            "id": text["_id"],
            "values": model.encode(text["chunk_text"]).tolist()
        }
    ])



{'_id': 'rec1', 'chunk_text': 'The Eiffel Tower was completed in 1889 and stands in Paris, France.', 'category': 'history'}
{'_id': 'rec2', 'chunk_text': 'Photosynthesis allows plants to convert sunlight into energy.', 'category': 'science'}
{'_id': 'rec3', 'chunk_text': 'Albert Einstein developed the theory of relativity.', 'category': 'science'}
{'_id': 'rec4', 'chunk_text': 'The mitochondrion is often called the powerhouse of the cell.', 'category': 'biology'}
{'_id': 'rec5', 'chunk_text': 'Shakespeare wrote many famous plays, including Hamlet and Macbeth.', 'category': 'literature'}
{'_id': 'rec6', 'chunk_text': 'Water boils at 100°C under standard atmospheric pressure.', 'category': 'physics'}
{'_id': 'rec7', 'chunk_text': 'The Great Wall of China was built to protect against invasions.', 'category': 'history'}
{'_id': 'rec8', 'chunk_text': 'Honey never spoils due to its low moisture content and acidity.', 'category': 'food science'}
{'_id': 'rec9', 'chunk_text': 'The speed of lig

In [ ]:
from bs4 import BeautifulSoup
import requests

base_url = "https://books.toscrape.com/catalogue/"
page_url = "category/books_1/index.html"

while page_url:
    url = base_url + page_url
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    books = soup.find_all('article', class_='product_pod')

    for book in books:
        href = book.h3.a['href']
        book_url = base_url + href.replace('../', '')
        book_response = requests.get(book_url)
        print(f"URL du livre: {book_url}")
        print(f"Statut de la réponse: {book_response.status_code}")
        book_soup = BeautifulSoup(book_response.text, 'html.parser')
        print(book_soup.prettify())

        title = book_soup.h1.text
        price = book_soup.find('p', class_='price_color').text
        description = book_soup.find('meta', attrs={'name': 'description'})['content'].strip()
        #availability = book_soup.find('p', class_='instock availability').text.strip()
        #rating = book_soup.find('p', class_='star-rating')['class'][1]

         # --- Insertion des données dans la base de données ---
        dense_index.upsert(vectors=[
            {
                "id": str(uuid.uuid4()),
                "values": model.encode(description).tolist(),
                "metadata": {
                    "title": title,
                    "price": price,
                }
            }
        ])

        print(f'Livre inséré : {title}')

    next_btn = soup.find('li', class_='next')
    if next_btn:
        page_url = next_btn.a['href']
    else:
        break

# --- Fermeture de la connexion ---
print("Scraping terminé. Toutes les données ont été sauvegardées dans books.db.")

## Prétraitement des données

In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# --- 1. Connexion à la base de données et chargement dans un DataFrame ---
# conn = sqlite3.connect('books.db')
# query = "SELECT * FROM books"
# df = pd.read_sql_query(query, conn)
# conn.close()

# Afficher les 5 premières lignes du DataFrame
print("DataFrame initial :")
print(df.head())
print("\n" + "="*50 + "\n")

# --- 2. Fonctions de prétraitement ---
# On va créer une fonction qui regroupe toutes les étapes
def preprocess_text(text):
    # 1. Normalisation : tout en minuscules
    text = text.lower()

    # 2. Nettoyage : suppression de la ponctuation, chiffres, etc.
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # 3. Tokenisation
    tokens = word_tokenize(text, language='english')


    # 4. Suppression des mots vides
    stop_words = set(stopwords.words('english'))

    tokens = [word for word in tokens if word not in stop_words]

    # On retourne une chaîne de caractères pour faciliter la vectorisation ultérieure
    return " ".join(tokens)



DataFrame initial :
                                   title    price  \
0                   A Light in the Attic  Â£51.77   
1                     Tipping the Velvet  Â£53.74   
2                             Soumission  Â£50.10   
3                          Sharp Objects  Â£47.82   
4  Sapiens: A Brief History of Humankind  Â£54.23   

                                         Description  
0  It's hard to imagine a world without A Light i...  
1  "Erotic and absorbing...Written with starling ...  
2  Dans une France assez proche de la nÃ´tre, un ...  
3  WICKED above her hipbone, GIRL across her hear...  
4  From a renowned historian comes a groundbreaki...  




In [ ]:
# --- 3. Application de la fonction à la colonne 'title' ---
# On crée une nouvelle colonne 'processed_title' pour stocker les résultats
data = df.copy()
titre = data.columns
for col in titre:
    if col in ['title', 'Description']:
        data[col] = data[col].apply(preprocess_text)

# Afficher les résultats
print("DataFrame avec les titres prétraités :")
print(data.head())

# Changement des types de données
# 1. Nettoyage de la colonne 'price'
# Remplacer d'abord le caractère d'encodage 'Â' (s'il existe)
df['price'] = df['price'].str.replace('Â', '', regex=False)
df['price'] = df['price'].str.replace('£', '', regex=False)
df['price'] = df['price'].str.replace(',', '.', regex=False) # Remplacer les virgules par des points si nécessaire
data['price'] = data['price'].astype(float)
data['Description'] = data['Description'].astype('string')
data['title'] = data['title'].astype('string')


# affichage
print("DataFrame avec les types de données modifiés :")
print(data.dtypes)
print("\n" + "="*50 + "\n")

DataFrame avec les titres prétraités :
                             title  price  \
0                      light attic  51.77   
1                   tipping velvet  53.74   
2                       soumission  50.10   
3                    sharp objects  47.82   
4  sapiens brief history humankind  54.23   

                                         Description  
0  hard imagine world without light attic nowclas...  
1  erotic absorbingwritten starling powerthe new ...  
2  dans une france assez proche de la ntre un hom...  
3  wicked hipbone girl across heart words like ro...  
4  renowned historian comes groundbreaking narrat...  
DataFrame avec les types de données modifiés :
title          string[python]
price                 float64
Description    string[python]
dtype: object




## Vectorization

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer


# 1. Combinaison des colonnes 'title' et 'Description'
data['combined_text'] = data['title'].fillna('') + ' ' + data['Description'].fillna('')

# 2. Vectorisation avec TF-IDF
# Créer un TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000) # Limiter le nombre de features pour l'efficacité

# Appliquer le vectoriseur à la nouvelle colonne 'combined_text'
tfidf_matrix = tfidf_vectorizer.fit_transform(data['combined_text'])

# Afficher la taille de la matrice vectorisée
# La forme est (nombre de documents, nombre de features)
print("Matrice TF-IDF pour les titres et descriptions combinés :")
print(tfidf_matrix.shape)

# Afficher les 5 premières lignes de la matrice (en format dense pour la lisibilité)
print("\nExemple de la matrice TF-IDF (5 premières lignes) :")
print(tfidf_matrix[:20].toarray())

## Clustering
### 1er test pour le clustering de textes

In [ ]:
import copy

In [ ]:
X = copy.deepcopy(tfidf_matrix)
wcss = []
for i in range(1, 11):
    modele_kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    modele_kmeans.fit(X)
    # inertia method returns wcss for that model
    wcss.append(modele_kmeans.inertia_)

In [ ]:
plt.figure(figsize=(12,6))
sns.lineplot(x = range(1, 11), y = wcss,marker='o',color='red')
plt.title('La méthode du coude')
plt.xlabel('Nombre de classe')